In [1]:
import torch
import os
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import json
import seaborn as sns
from PIL import Image
import copy
from collections import OrderedDict
from torch.optim import lr_scheduler
from torch.autograd import Variable
import operator
from torch.nn import init
import math
import torch.utils.model_zoo as model_zoo


starttime = time.time()

def random_seeding(seed_value, on_gpu):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    if on_gpu: torch.cuda.manual_seed_all(seed_value) # gpu vars

    
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU.....Left, Square, Square, Square, Left ')
    
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")


CUDA is available!  Training on GPU.....Left, Square, Square, Square, Left 
Time Stage: 0.111 seconds


In [2]:
#break

In [3]:
#random_seeding(64,train_on_gpu)

In [4]:
def plotda_training(train_losses, valid_losses):
    #plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training loss')
    plt.plot(valid_losses, label='Validation loss')
    plt.xlabel('epochs')
    plt.legend(frameon=False)
    plt.title('Loss progress')
    ax.set_yscale('log')
    plt.figure(figsize=(15,24))
    plt.show()
    plt.savefig('progress.png')

In [5]:
def plotda_learning(scheduler_steps):
    #plt.subplot(1, 2, 1)
    plt.plot(scheduler_steps, label='LearningRate')
    plt.xlabel('epochs')
    plt.legend(frameon=False)
    plt.title('LearningRate progress')  
    ax.set_yscale('log')
    plt.figure(figsize=(15,24))
    plt.show()
    plt.savefig('LearningRate.png')

In [6]:
# define training and test data directories
data_dir = 'C:/PyTorch/flower_data/'
#data_dir = 'C:/PyTorch/flower_photos/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

dirs = {'train': train_dir, 
        'valid': valid_dir, 
        'test' : test_dir}

In [7]:
# load and transform data using ImageFolder

# VGG-16 Takes 224x224 images as input, so we resize all of them
train_transforms = transforms.Compose([transforms.RandomRotation(64),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# print out some data stats
print('Num training images: ', len(train_data))
print('Num valid images: ', len(valid_data))
print('Num test images: ', len(test_data))

# define dataloader parameters
batch_size = 130 #130 #30 #18
num_workers= 8
imagestoshow = 16#20#16#8
n_epochs1 = 7
n_epochs2 = 2
lr1min = 1e-08
lr1max = .006
lr2min = 1e-08
lr2max = 0.00008

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

Num training images:  6552
Num valid images:  832
Num test images:  816
Time Stage: 0.283 seconds


In [8]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
class_names = train_data.classes

sortedclasses = sorted(cat_to_name.items())
print(sortedclasses[0])



('1', 'pink primrose')


### Visualize a Batch of Training Data

In [9]:
# helper function to un-normalize and display an image
def imshow(img):

    img = img.transpose(1,2,0)
    img = img * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    img = img.clip(0, 1)

    plt.imshow(img)  # convert from Tensor image

# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(15, 12))
# display 20 images
for idx in np.arange(imagestoshow):
    ax = fig.add_subplot(4, imagestoshow/4, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(sortedclasses[labels[idx]])
    
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")    

In [10]:
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")
# Load the pretrained model from pytorch
model = models.resnet18(pretrained=True)
#print(model)


#model = xception(True)
# Freeze training for all "features" layers
for param in model.parameters():
    param.requires_grad = False
    
    
#n_inputs = model.classifier.in_features
n_inputs = model.fc.in_features
#print(model.classifier.in_features) 
#print(model.classifier.out_features) 
int(round(n_inputs*0.8))

      
class ClassifierNew(nn.Module):
    def __init__(self, inp = 512, h1=1024, out = 102, d=0.35):
        super().__init__()
        self.ap = nn.AdaptiveAvgPool1d(512)
        self.mp = nn.AdaptiveMaxPool1d(512)
        self.bn0 = nn.BatchNorm1d(512*2,eps=1e-05, momentum=0.1, affine=True)
        self.dropout0 = nn.Dropout(d)
        self.fc1 = nn.Linear(512*2, 500)
        self.bn1 = nn.BatchNorm1d(h1,eps=1e-05, momentum=0.1, affine=True)
        self.dropout1 = nn.Dropout(d)
        self.fc2 = nn.Linear(500, out)
        
    def forward(self, x):
        ap = self.ap(x)
        mp = self.mp(x)
        x = torch.cat((ap,mp),dim=1)
        x = x.view(-1, 512*2)
        x = self.bn0(x)
        x = self.dropout0(x)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = self.dropout1(x)         
        x = self.fc2(x)
        
        return x

#model.classifier = classifier
model.fc = ClassifierNew()
print(model)



# if GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()

criterion = nn.NLLLoss()


# specify optimizer (stochastic gradient descent) and learning rate = 0.001
#optimizer = optim.Adam(model.parameters(), lr=0.0006, betas=(0.9, 0.999), eps=1e-08, 
#                       weight_decay=1e-05, amsgrad=True)
optimizer = optim.Adam(model.fc.parameters(), lr=lr1max, betas=(0.9, 0.999), eps=1e-08, 
                       weight_decay=1e-04, amsgrad=True)
#optimizer = optim.Adam(model.parameters(), lr=lr1max, betas=(0.9, 0.999), eps=1e-08, 
#                       weight_decay=1e-04, amsgrad=True)
#optimizer = optim.Adam(model.classifier.parameters(), lr=lr1max, betas=(0.9, 0.999), eps=1e-08, 
#                       weight_decay=1e-04, amsgrad=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.10, patience=4, 
                                                 verbose=False, threshold=0.006, threshold_mode='rel', 
                                                 cooldown=0, min_lr=0, eps=1e-08)
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

Time Stage Start: 0.305 seconds
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu):

In [11]:
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")


valid_loss_min = np.Inf # track change in validation loss

train_losses = []
valid_losses = []
scheduler_steps = []

for epoch in range(1, n_epochs1+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    start = time.time()
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
    #    if batch_idx % 20 == 19:    # print training loss every specified number of mini-batches
     #       print('\t Epoch %d, Batch %d loss: %.16f' %
      #            (epoch, batch_idx + 1, train_loss / 20))
       #     print(f"\t \tTime per batch: {(time.time() - start)/3:.3f} seconds")
        #    train_loss = 0.0
        
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    scheduler.step(valid_loss)
    scheduler_steps.append(optimizer.param_groups[0]['lr'])
    if optimizer.param_groups[0]['lr'] <= lr1min:
        optimizer.param_groups[0]['lr']=lr1max
        
    #print('lr: {} '.format(optimizer.param_groups[0]['lr']))   
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tlr: {} '.format(
        epoch, train_loss, valid_loss, optimizer.param_groups[0]['lr']))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        #torch.save(model.state_dict(), 'model_augmented.pt')
        model.class_to_idx = train_data.class_to_idx
        #model.cpu()
        torch.save({'state_dict': model.state_dict(), 
                    'class_to_idx': model.class_to_idx}, 
                    'Project_WarthogRevMv2.pth')
        valid_loss_min = valid_loss
    
    print(f"Time per epoch: {(time.time() - start):.3f} seconds")
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

Time Stage Start: 2.445 seconds


RuntimeError: Expected 3-dimensional tensor, but got 2-dimensional tensor for argument #1 'self' (while checking arguments for adaptive_avg_pool1d)

In [ ]:
plotda_training(train_losses,valid_losses)
plotda_learning(scheduler_steps)

In [ ]:
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

model.eval()
# iterate over test data
for batch_idx, (data, target) in enumerate(test_loader):
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    #for i in range(20):
    for i in range (len(target.data)): 
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))


for i in range(102):
   if class_total[i] > 0:
       print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
           sortedclasses[i], 100 * class_correct[i] / class_total[i],
           np.sum(class_correct[i]), np.sum(class_total[i])))
   else:
       print('Test Accuracy of %5s: N/A (no training examples)' % (i))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
   100. * np.sum(class_correct) / np.sum(class_total),
   np.sum(class_correct), np.sum(class_total)))
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

if train_on_gpu:
    images = images.cpu()
images = images.numpy()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(15, 16))
for idx in np.arange(imagestoshow):
    ax = fig.add_subplot(4, imagestoshow/4, idx+1, xticks=[], yticks=[])
    #plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    imshow(images[idx])
    ax.set_title("{}\n {}".format(sortedclasses[preds[idx]], sortedclasses[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)

    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']

    return model


model = load_checkpoint('Project_WarthogRevMv2.pth')

for param in model.parameters():
    param.requires_grad = True
    
#optimizer = optim.Adam(model.parameters(), lr=0.00008, betas=(0.9, 0.999), eps=1e-08, 
#                       weight_decay=1e-04, amsgrad=True)
optimizer = optim.Adam(model.parameters(), lr=lr2max, betas=(0.9, 0.999), eps=1e-08, 
                       weight_decay=1e-04, amsgrad=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.10, patience=4, 
                                                 verbose=False, threshold=0.0008, threshold_mode='rel', 
                                                 cooldown=0, min_lr=0, eps=1e-08)
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")
# number of epochs to train the model


valid_loss_min = np.Inf # track change in validation loss
train_losses2 = []
valid_losses2 = []

for epoch in range(1, n_epochs2+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    start = time.time()
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
    #    if batch_idx % 20 == 19:    # print training loss every specified number of mini-batches
     #       print('\t Epoch %d, Batch %d loss: %.16f' %
      #            (epoch, batch_idx + 1, train_loss / 20))
       #     print(f"\t \tTime per batch: {(time.time() - start)/3:.3f} seconds")
        #    train_loss = 0.0
        
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(valid_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    scheduler.step(valid_loss)
    train_losses2.append(train_loss)
    valid_losses2.append(valid_loss)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    scheduler_steps.append(optimizer.param_groups[0]['lr'])
    if optimizer.param_groups[0]['lr'] <= lr2min:
        optimizer.param_groups[0]['lr']=lr2max
    #print('lr: {} '.format(optimizer.param_groups[0]['lr']))   
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tlr: {} '.format(
        epoch, train_loss, valid_loss, optimizer.param_groups[0]['lr']))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        #torch.save(model.state_dict(), 'model_augmented.pt')
        model.class_to_idx = train_data.class_to_idx
        #model.cpu()
        torch.save({'state_dict': model.state_dict(), 
                    'class_to_idx': model.class_to_idx}, 
                    'Project_WarthogRevMv2True.pth')
        valid_loss_min = valid_loss
    
    print(f"Time per epoch: {(time.time() - start):.3f} seconds")
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
plotda_training(train_losses,valid_losses)
plotda_training(train_losses2,valid_losses2)
plotda_learning(scheduler_steps)

In [ ]:
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

model.eval()
# iterate over test data
for batch_idx, (data, target) in enumerate(test_loader):
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    #for i in range(20):
    for i in range (len(target.data)): 
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))


for i in range(102):
   if class_total[i] > 0:
       print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
           sortedclasses[i], 100 * class_correct[i] / class_total[i],
           np.sum(class_correct[i]), np.sum(class_total[i])))
   else:
       print('Test Accuracy of %5s: N/A (no training examples)' % (i))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
   100. * np.sum(class_correct) / np.sum(class_total),
   np.sum(class_correct), np.sum(class_total)))
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")



In [ ]:
print(f"Time Stage Start: {(time.time() - starttime):.3f} seconds")
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

if train_on_gpu:
    images = images.cpu()
images = images.numpy()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(15, 16))
for idx in np.arange(imagestoshow):
    ax = fig.add_subplot(4, imagestoshow/4, idx+1, xticks=[], yticks=[])
    #plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    imshow(images[idx])
    ax.set_title("{}\n {}".format(sortedclasses[preds[idx]], sortedclasses[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
image_pathtet = os.path.join(data_dir, 'test/98/image_07758.jpg')
#imgtet = Image.open(image_pathtet)
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
def process_image(image_path):
    ''' 
    Scales, crops, and normalizes a PIL image for a PyTorch       
    model, returns an Numpy array
    '''
    # Open the image
    img = Image.open(image_path)

    # Resize
    if img.size[0] > img.size[1]:
        img.thumbnail((5000, 256))
    else:
        img.thumbnail((256, 5000))

    # Crop 
    left_margin = (img.width-224)/2
    bottom_margin = (img.height-224)/2
    right_margin = left_margin + 224
    top_margin = bottom_margin + 224
    img = img.crop((left_margin, bottom_margin, right_margin, top_margin))
    
    # Normalize
    img = np.array(img)/255
    mean = np.array([0.485, 0.456, 0.406]) #Required mean
    std = np.array([0.229, 0.224, 0.225]) #Required std
    img = (img - mean)/std
    
    # Move color channels to first dimension as expected by PyTorch
    #img = img.numpy().transpose((2, 0, 1))
    img = img.transpose((2, 0, 1))
    
    return img


In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
process_image(image_pathtet)
imshow(process_image(image_pathtet))
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
        # Process image
    img = process_image(image_path)
    
    # Numpy -> Tensor
    image_tensor = torch.from_numpy(img).type(torch.FloatTensor)

    # Add batch of size 1 to image
    model_input = image_tensor.unsqueeze(0)
    
    # Probs
    probs = torch.exp(model.forward(model_input))
    
    # Top probs
    top_probs, top_labs = probs.topk(topk)
    top_probs = top_probs.detach().numpy().tolist()[0] 
    top_labs = top_labs.detach().numpy().tolist()[0]
    
    # Convert indices to classes
    idx_to_class = {val: key for key, val in    
                                      model.class_to_idx.items()}

    top_labels = [idx_to_class[lab] for lab in top_labs]
    top_flowers = [cat_to_name[idx_to_class[lab]] for lab in top_labs]

    return top_probs, top_labels, top_flowers

In [ ]:
probs, classes, flowers = predict(image_pathtet, model.cpu())
print(probs)
print(classes)
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")

In [ ]:
# TODO: Display an image along with the top 5 classes
def plot_solution(image_path, model):
    # Set up plot
    plt.figure(figsize = (6,10))
    ax = plt.subplot(2,1,1)

    # Set up title
    flower_num = image_path.split('/')[4]
    flowerClass = cat_to_name[flower_num]
    #print(flower_num)
    #print(flowerClass)
    # Plot flower
    img = process_image(image_path)
    #imshow(img, ax, title = title_);
    imshow(img, ax);
    #ax.set_title(flowerClass, color=("green"))


    # Make prediction
    probs, labs, flowers = predict(image_path, model) 
    
    ax.set_title("{}-{}".format(flower_num, flowerClass), 
                 color=("green" if flower_num==labs[0] else "red"))

    # Plot bar chart
    plt.subplot(2,1,2)
    sns.barplot(x=probs, y=flowers, color=sns.color_palette()[0]);
    plt.show()

In [ ]:
plot_solution(image_pathtet, model)
probs, labs, flowers = predict(image_pathtet, model)
print(probs)
print(labs)
print(flowers)
print(f"Time Stage: {(time.time() - starttime):.3f} seconds")